# Ornstein-Uhlenbeck Process Parameter Estimation

## Relation to $\text{AR}(1)$

Assume $X_t$ is known. From equation $(4)$ the distribution of $t+\Delta t$? can be found. It follows that,

$
\begin{align}
X_{t+\Delta t} =  X_t e^{-\lambda \Delta t} + \mu^* \left( 1 - e^{-\lambda \Delta t} \right) + \sqrt{ \frac{\sigma^2}{2\lambda} \left( 1 - e^{-2\lambda \Delta t} \right)} \hspace{5pt} \varepsilon_t
\end{align}
$

Now, $\text{AR}(1)$ with constant offset can be written as,

$
\begin{align}
X_t = \hat{\varphi} X_{t-1} + \hat{\mu} + \hat{\sigma} \varepsilon_t
\end{align}
$

Comparing with the previous equation gives,

$
\begin{align}
&\hat{\varphi} = e^{-\lambda \Delta t} \\
&\hat{\mu} = \mu^* \left(1 - e^{-\lambda \Delta t} \right) \\
&\hat{\sigma} = \sqrt{\frac{\sigma^2}{2\lambda}\left(1 - e^{-2\lambda \Delta t}\right)}
\end{align}
$

Further if it is assumed that $\Delta t \ll 1$, it follows that $e^{-\lambda \Delta t} \approx 1 - \lambda \Delta t$, and

$
\begin{align}
&\hat{\varphi} \approx 1 - \lambda \Delta t \\
&\hat{\mu} \approx \mu^* \Delta t \\
&\hat{\sigma} \approx \sigma \sqrt{\Delta t}
\end{align}
$

In terms of the Ornstein-Uhlenbeck parameters $\text{AR}(1)$ can be written as,

$
\begin{align}
X_t \approx \left( 1 - \lambda \Delta t \right)X_{t-\Delta t} + \mu^* \Delta t + \sigma \sqrt{\Delta t}\hspace{2pt} \varepsilon_t \\
\end{align}
$

Letting $\Delta t = 1$ recovers $\text{AR}(1)$ with Ornstein-Uhlenbeck parameters, 

$
\begin{align}
X_t = \left( 1 - \lambda \right)X_{t-1} + \mu^* + \sigma \varepsilon_t \\
\end{align}
$

## Parameter Estimation

The Ornstein-Uhlenbeck model parameters $\lambda$, $\mu^*$ and $\sigma$ can be evaluated as a function $\Delta t$ using the relations with $\text{AR}(1)$ parameters</br>
previously discussed,

$
\begin{align}
&\hat{\varphi} = e^{-\lambda \Delta t} \\
&\hat{\mu} = \mu^* \left(1 - e^{-\lambda \Delta t} \right) \\
&\hat{\sigma} = \sqrt{\frac{\sigma^2}{2\lambda}\left(1 - e^{-2\lambda \Delta t}\right)}
\end{align}
$

The companion $\text{AR}(1)$ process is given by,

$
\begin{align}
X_t = \hat{\varphi} X_{t-1} + \hat{\mu} + \hat{\sigma} \varepsilon_t
\end{align}
$

OLS can be used to determine $\hat{\varphi}$, $\hat{\mu}$ and $\hat{\sigma}$ from the available data. Then $\lambda$ is given by,

$
\begin{align}
\lambda = - \frac{\ln{\hat{\varphi}}}{\Delta t}
\end{align}
$

Substitution into the equation for $\mu^*$ gives,

$
\begin{align}
\mu^* = \frac{\hat{\mu}}{1 - \hat{\varphi}}
\end{align}
$

which is the previously obtained relation for $t \to \infty$. Finally, 

$
\begin{align}
\sigma = \sqrt{\frac{2\lambda\hat{\sigma}^2}{1 - \hat{\varphi}^2}}
\end{align}
$

